In [1]:
import pandas as pd
from pandas import option_context

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import re
import string

## Main notebook for pre-processing tweets

### Prepare dataframe

In [2]:
df = pd.read_csv('custom_out2.csv')
df.dropna(axis=1,inplace=True)

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df['year'] = pd.DatetimeIndex(df['date']).year

## Filter down to non-Revolut tweets (intention is to understand tweets from customers)
df_eng = df[(df['username'] != 'revolutapp') & (df['language'] == 'en') & (df['year'] == 2020)]
df_eng.shape

(110317, 23)

### Clean tweet column

In [4]:
def clean_text(text):
    '''Make text lowercase, remove punctuation, remove reference to @revolutapp,remove links and mentions'''
    text = text.lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub('[0-9\n]',' ',text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('revolutapp', '', text)
    text = text.strip()
    text = re.sub("\s\s+", " ", text)
    
    return text

df_eng['tweet'] = df_eng.tweet.map(clean_text)

<ipython-input-4-0f67e7548dfc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['tweet'] = df_eng.tweet.map(clean_text)


In [5]:
## Remove emojis
df_eng['tweet'] = df_eng['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

<ipython-input-5-77c184c069e8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_eng['tweet'] = df_eng['tweet'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))


In [6]:
df_eng.shape

(110317, 23)

In [7]:
with option_context('display.max_colwidth', 600):
    display(df_eng.sample(5))

,id,conversation_id,created_at,date,time,timezone,user_id,username,tweet,language,...,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,video,reply_to,year
64472,1320766618620682240,1320713422242283520,2020-10-26 12:37:55 EDT,2020-10-26,12:37:55,-500,848296698732584964,swagger5942,unlock my account,en,...,0,0,0,[],[],https://twitter.com/Swagger5942/status/1320766618620682240,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]",2020
209181,1229454363229380610,1229426334625095680,2020-02-17 12:15:37 EST,2020-02-17,12:15:37,-500,4102562173,dimitre_sarkis,want a banking nightmare sign up now if this is the future of banking it is bleak,en,...,0,0,2,[],[],https://twitter.com/dimitre_sarkis/status/1229454363229380610,False,0,"[{'screen_name': 'NeverLoosehop', 'name': 'Innovations_Follower', 'id': '49395579'}, {'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]",2020
49980,1329466433051521026,1329466433051521026,2020-11-19 11:47:52 EST,2020-11-19,11:47:52,-500,1154352109271298049,teaistriumphh,loooool wanting to charge me to get a new card delivered in weeks you okay,en,...,1,0,0,[],[],https://twitter.com/teaistriumphh/status/1329466433051521026,False,0,[],2020
142229,1270745058896158720,1270742646865133572,2020-06-10 11:50:05 EDT,2020-06-10,11:50:05,-500,2800778152,ruben_prod,i was,en,...,1,0,0,[],[],https://twitter.com/ruben_prod/status/1270745058896158720,False,0,"[{'screen_name': 'RevolutApp', 'name': 'Revolut', 'id': '2676313880'}]",2020
179003,1247819887906562048,1247819887906562048,2020-04-08 05:33:39 EDT,2020-04-08,05:33:39,-500,17067737,zigojacko,do you guys have anyone working at the moment our business application has been awaiting review for ages now thanks,en,...,1,0,0,[],[],https://twitter.com/zigojacko/status/1247819887906562048,False,0,[],2020


In [8]:
df_eng = df_eng[df_eng['tweet'].map(lambda x: len((x)) > 22)]

In [9]:
df_eng.shape

(97065, 23)

In [10]:
df_eng = df_eng[(df_eng['username'] != 'bunq') &
                (df_eng['username'] != 'financialombuds') &
                (df_eng['username'] != 'barclaysukhelp') &
                (df_eng['username'] != 'stxfn') &
                (df_eng['username'] != 'intokpyouseeme') &
                (df_eng['username'] != 'imaginecurve') &
                (df_eng['username'] != 'monzo') &
                (df_eng['username'] != 'davymc82') &
                (df_eng['username'] != 'revolutbusiness') &
                (df_eng['username'] != 'hola_mide')&
                (df_eng['username'] != 'adegbenrotoni')&
                (df_eng['username'] != 'pensionbee')&
                (df_eng['username'] != 'markdampier')&
                (df_eng['username'] != 'xrpthefuture')&
                (df_eng['username'] != 'lloydssmart')&
                (df_eng['username'] != 'cashquestions') &
                (df_eng['username'] != 'uniquity_co')&
                (df_eng['username'] != 'timipapi')&
                (df_eng['username'] != 'thegarrick_')&
                (df_eng['username'] != 'ukay_gold')&
                (df_eng['username'] != 'tori_okoye') &
                (df_eng['username'] != 'snoopapp')&
                (df_eng['username'] != 'oluwaseeto_')&
                (df_eng['username'] != 'pboom2020')&
                (df_eng['username'] != 'gbemss_')&
                (df_eng['username'] != 'syncnd')&
                (df_eng['username'] != 'chrismaurice')&
                (df_eng['username'] != 'adetolaov')&
                (df_eng['username'] != '_ballofyarn')&
                (df_eng['username'] != 'emmanuel_agbai_')&
                (df_eng['username'] != 'cobalt_dl')&
                (df_eng['username'] != 'ad3bay0')&
                (df_eng['username'] != 'dadaben_')&
                (df_eng['username'] != 'stedas')&
                (df_eng['username'] != 'thekitoye')&
                (df_eng['username'] != 'allofsimeon')&
                (df_eng['username'] != 'bola_ware')&
                (df_eng['username'] != 'namzo')&
                (df_eng['username'] != 'iamayomidotun')&
                (df_eng['username'] != 'littlespender')&
                (df_eng['username'] != 'consistentbenny')&
                (df_eng['username'] != 'joe_of_the_90')&
                (df_eng['username'] != 'eltitocharlie')&
                (df_eng['username'] != 'marklisseman')&
                (df_eng['username'] != 'samabrika')&
                (df_eng['username'] != 'alex_lucas18')&
                (df_eng['username'] != 'pandelisz')&
                (df_eng['username'] != 'applepaydude')&
                (df_eng['username'] != 'iamthewalrus28') &
                (df_eng['username'] != 'imrelb') &
                (df_eng['username'] != 'tomislavcapan') &
                (df_eng['username'] != 'cryptdigital') &
                (df_eng['username'] != 'alexfintec') &
                (df_eng['username'] != 'saleslocker') &
                (df_eng['username'] != 'lexsokolin') &
                (df_eng['username'] != 'greg_bremner') &
                (df_eng['username'] != 'revolutexposed2') &
                (df_eng['username'] != 'millie1703') &
                (df_eng['username'] != 'clinton131313') &
                (df_eng['username'] != 'profpinar') &
                (df_eng['username'] != 'mr_twelve_12') &
                (df_eng['username'] != 'stvnmyls') &
                (df_eng['username'] != 'moreyogaldn') &
                (df_eng['username'] != 'lukemckibben') &
                (df_eng['username'] != 'jonkelly31') &
                (df_eng['username'] != 'kirsikkalar') &
                (df_eng['username'] != 'mr_c_mr__c') &
                (df_eng['username'] != 'xrpkahn') &
                (df_eng['username'] != 'karldarby20karl') &
                (df_eng['username'] != 'digital_chris') &
                (df_eng['username'] != 'lionel46294550') &
                (df_eng['username'] != 'techdirect79')] 

In [11]:
df_eng.shape

(96506, 23)

In [12]:
df_eng = df_eng[(df_eng['tweet'] != 'thank you ive responded') &
                   (df_eng['tweet'] != 'iam still waiting for u') &
                   (df_eng['tweet'] != 'thank u well appreciated') &
                   (df_eng['tweet'] != 'nothing on dm im waiting') &
                   (df_eng['tweet'] != 'just wanted to let you know') &
                   (df_eng['tweet'] != 'yo they took it down booooo') &
                   (df_eng['tweet'] != 'they havent responded at all') &
                   (df_eng['tweet'] != 'ok thanks just replied to dm') &
                   (df_eng['tweet'] != 'found more including and ') &
                   (df_eng['tweet'] != 'what does that mean exactly') &
                   (df_eng['tweet'] != 'im not seeing a message') &
                   (df_eng['tweet'] != 'just wanted to let you know') &
                   (df_eng['tweet'] != 'any sign of a update yet') &
                   (df_eng['tweet'] != 'why cant you write to me dm') &
                   (df_eng['tweet'] != 'following when will you contact me') &
                   (df_eng['tweet'] != 'we all deserve our tiny bit') &
                   (df_eng['tweet'] != 'just followed thank you') &
                   (df_eng['tweet'] != 'well well welllook what arrived') &
                   (df_eng['tweet'] != 'ill post anything though') &
                   (df_eng['tweet'] != 'thanks a mil following now') &
                   (df_eng['tweet'] != 'who needs to know already knows') &
                   (df_eng['tweet'] != 'i hope to stay informed') &
                   (df_eng['tweet'] != 'ok thanks for the prompt reply') &
                   (df_eng['tweet'] != 'its been two days since') &
                   (df_eng['tweet'] != 'i ahould add or potterrichard') &
                   (df_eng['tweet'] != 'are you prepared for it') &
                   (df_eng['tweet'] != 'thank you now following x') &
                   (df_eng['tweet'] != 'i use a mbits cable internet connection') &
                   (df_eng['tweet'] != 'take lessons from these people') &
                   (df_eng['tweet'] != 'thanks thats good to know') &
                   (df_eng['tweet'] != 'thanks i followed you just now') &
                   (df_eng['tweet'] != 'thank you very much just replied') &
                   (df_eng['tweet'] != 'online research motley fool is decent') &
                   (df_eng['tweet'] != 'not that jd nio snap square pinterest down but strong last week stitch fix') &
                   (df_eng['tweet'] != 'best way to contact them') &
                   (df_eng['tweet'] != 'i have replied to the dm') &
                   (df_eng['tweet'] != 'ok done can you dm me please') &
                   (df_eng['tweet'] != 'i have just read the dm and replied') &
                   (df_eng['tweet'] != 'no im done tbh pain in the arse it is') &
                   (df_eng['tweet'] != 'great thanks will do now') &
                   (df_eng['tweet'] != 'yes i am following now what is dms please') &
                   (df_eng['tweet'] != 'whats up gone all shy suddenly') &
                   (df_eng['tweet'] != 'no lets do it here in the open') &
                   (df_eng['tweet'] != 'problem solved thank you') &
                   (df_eng['tweet'] != 'thanks for clarifying that') &
                   (df_eng['tweet'] != 'thanks just followed you now') &
                   (df_eng['tweet'] != 'i just got one of those too') &
                   (df_eng['tweet'] != 'followed but i cant dm you') &
                   (df_eng['tweet'] != 'hello any update please') &
                   (df_eng['tweet'] != 'just tried to use them and') &
                   (df_eng['tweet'] != 'knows about this im sure') &
                   (df_eng['tweet'] != 'worked a treat many thanks') &
                   (df_eng['tweet'] != 'ready im already following you') &
                   (df_eng['tweet'] != 'ok thank you i will do that') &
                   (df_eng['tweet'] != 'can someone dm me please') &
                   (df_eng['tweet'] != 'just to het onboard now') &
                   (df_eng['tweet'] != 'you can delete that tweet now') &
                   (df_eng['tweet'] != 'time for to respond now') &
                   (df_eng['tweet'] != 'yes hear that whats stopping you') &
                   (df_eng['tweet'] != 'would like to know this too') &
                   (df_eng['tweet'] != 'great thanks for the reply') &
                   (df_eng['tweet'] != 'same aha account blocked') &
                   (df_eng['tweet'] != 'same for me bro go to if you can') &
                   (df_eng['tweet'] != 'pleaseresponddd')]

In [13]:
df_eng.shape

(96446, 23)

#### Correct misspelled words noticed in data review process

In [14]:
remove_misspelled = (lambda x: 
                 x.replace('seriusly', 'seriously')
                 .replace('incomewhich', 'income which')
                 .replace('revolutuserswantscrypto', 'revolut users want crypto')
                 .replace('chatwhere', 'chat where')
                 .replace('loungesthat', 'lounges that')
                 .replace('relovut', 'revolut')
                 .replace('sitesstill', 'site still')
                 .replace('moneylaundring', 'money laundering')
                 .replace('changeslogin', 'changes login')
                 .replace('transactioned', 'transaction')
                 .replace('moneygood', 'money good')
                 .replace('premim', 'premium')
                 .replace('resetfrustrating', 'reset frustrating')
                 .replace('moneywithout', 'money without')
                 .replace('exchanses', 'exchanges')
                 .replace('plattform', 'platform')
                 .replace('informaton', 'information')
                 .replace('generatingsaving', 'generating savings')
                 .replace('customerservicemissing', 'customer service missing')
                 .replace('verifiacationcould', 'verification could')
                 .replace('regulatinons', 'regulations')
                 .replace('fraudulence', 'fraudulent')
                 .replace('imsurqnce', 'insurance')
                 .replace('accountmoney', 'account money')
                 .replace('reccomand', 'recommend')
                 .replace('billsrent', 'bills rent')
                 .replace('badcustomerservice', 'bad customer service')
                 .replace('idverification', 'id verification')
                 .replace('wault', 'vault')
                 .replace('infos ', 'information ')
                 .replace('info ', 'information ')
                 .replace('revoiut', 'revolut')
                 .replace('buyinvest', 'buy invest')
                 .replace('criptos', 'crypto')
                 .replace('robindhood', 'robinhood')
                 .replace('appfrozen', 'app frozen')
                 .replace('rainbowrevolutcard', 'rainbow revolut card')
                 .replace('refferral', 'referral')
                 .replace('varification', 'verification')
                 .replace('accouts', 'accounts')
                 .replace('autocloses', 'auto closes'))

df_eng['tweet'] = df_eng.tweet.map(remove_misspelled)

In [15]:
def remove_space(text):
    text = text.strip()
    return text
    
df_eng['tweet'] = df_eng.tweet.map(remove_space)

In [17]:
df_eng.shape

(96446, 23)

In [16]:
df_eng.to_csv('train.csv')

In [24]:
tweets = [tweet for tweet in df_eng['tweet']]

### Lemmatize, Tokenize and count words

Note that this is separate to tokenization above - what is correct in terms of process?

In [25]:
from nltk.util import ngrams

from collections import Counter

from nltk.corpus import stopwords

from textblob import TextBlob

In [26]:
stop = stopwords.words('english')
stop.extend(['ive', 'im', 'itd', 'youre', 'hi', 'hello', 'hey', 'eg', 'l', 'h','w','v','u'])

In [28]:
lemma = nltk.stem.WordNetLemmatizer()
counter = Counter()

for tweet in df_eng['tweet']:
    tweet = lemma.lemmatize(tweet)
    tweet = TextBlob(tweet).words  # tokenize words
    tweet = [w for w in tweet if w not in stop]   
    counter += Counter(tweet)
    
len(counter)

In [31]:
for phrase, count in counter.most_common(10):
    print('%20s %i' % ("".join(phrase), count))

             account 30834
             revolut 17982
                 app 16720
               money 15512
              please 13324
                help 12066
                 get 10704
                cant 10033
                chat 9365
                card 8423
